# Klassifikation mit einem Neuronalen Netz

## Trainingsdaten

In [1]:
!pip install torch
!pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/e4/cf/8f9305cc0ea26badbbb3558ecae54c04a245429f03168f7fad502f8a5b25/torchvision-0.22.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torch==2.7.0 from https://files.pythonhosted.org/packages/13/85/6c1092d4b06c3db1ed23d4106488750917156af0b24ab0a2d9951830b0e9/torch-2.7.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf356191/sympy-1.14.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.7 MB 2.9 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.7 MB 2.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.4/1.7 MB 2.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.5/1.7 MB 2.7 MB/s eta 0:0

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'C:\\Users\\Wiebke Petersen\\anaconda3\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# load mnist dataset from pytorch
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transforms.ToTensor())



ModuleNotFoundError: No module named 'torchvision'

In [ ]:
train_data.data.shape, test_data.data.shape

Wir haben 60.000 Trainingsbilder (jeweils 28x28 Pixel) und 10.000 Testbilder im Datenset MNIST 

In [ ]:
print(train_data.data[17].shape)
print(train_data.targets[17])
print(train_data.data[17][5:10][:])



In [ ]:
# visualize the 17th image
plt.imshow(train_data.data[17], cmap='gray')

## Definition des Neuronalen Netzes

In [ ]:
# set parameters for feed forward neural network
input_size = 28*28 #=784
hidden_size = 400
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# create dataloader
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

# create neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) # Hidden layer 
        self.relu = nn.ReLU() # Activation function
        self.l2 = nn.Linear(hidden_size, num_classes) # Output layer
        
    def forward(self, x):  # Forward pass
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)


In [ ]:
# test the model before training on images 17:20 from test data
# print predictions and images

for i in range(17, 21):
    img = test_data.data[i]
    plt.imshow(img, cmap='gray')
    plt.show()
    img = img.reshape(-1, 28*28)
    img = img.type(torch.FloatTensor)
    output = model(img)
    _, predicted = torch.max(output.data, 1)
    print('Predicted: ', predicted.item())
        

Wie erwartet sind die Gewichte des Neuronalen Netzes vor dem Training zufällig gewählt, so dass die Klassifikation misslingt.

## Training des Neuronalen Netz

In [ ]:

# loss and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward and optimize
        optimizer.zero_grad() # clear gradients
        loss.backward() # backpropagation
        optimizer.step() # update weights
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))



## Test des Neuronalen Netzes

In [ ]:
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


In [ ]:
# test the model after training on images 17:20 from test data
# print predictions and images

for i in range(17, 21):
    img = test_data.data[i]
    plt.imshow(img, cmap='gray')
    plt.show()
    img = img.reshape(-1, 28*28)
    img = img.type(torch.FloatTensor)
    output = model(img)
    _, predicted = torch.max(output.data, 1)
    print('Predicted: ', predicted.item())

## Test mit kleinerem Trainingsdatensatz 

In [ ]:
# choose a small traings set randomly
train_data_small = datasets.MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
train_data_small.data = train_data_small.data[:70]
train_data_small.targets = train_data_small.targets[:70]



In [ ]:
train_data, train_data_small

In [ ]:
import matplotlib.pyplot as plt



tr = train_data_small.targets.numpy()

# plot bar chart of the number of images in each class
plt.bar(np.arange(10), np.bincount(tr))



In [ ]:
# train model_small on train_data_small
# print loss and accuracy on train_data_small and test_data_small

batch_size = 10
num_epochs = 10
learning_rate = 0.001
report = 5

model_small = NeuralNet(input_size, hidden_size, num_classes)
 


# create dataloader
train_loader_small = DataLoader(dataset=train_data_small, batch_size=batch_size, shuffle=True)
test_loader_small = DataLoader(dataset=test_data_small, batch_size=batch_size, shuffle=False)


# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_small.parameters(), lr=learning_rate)

# train the model
total_step = len(train_loader_small)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader_small):
        # reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # forward pass
        outputs = model_small(images)
        loss = criterion(outputs, labels)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print loss on train_data_small and test_data_small 
        if (i+1) % report == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        # print accuracy on train_data_small and test_data_small
        if (i+1) % report == 0:
            # In test phase, we don't need to compute gradients (for memory efficiency)
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in train_loader_small:
                    images = images.reshape(-1, 28*28)
                    outputs = model_small(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Train Accuracy of the model on the train images: {} %'.format(100 * correct / total))
            
            # In test phase, we don't need to compute gradients (for memory efficiency)
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader_small:
                    images = images.reshape(-1, 28*28)
                    outputs = model_small(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
            
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader_small:
        images = images.reshape(-1, 28*28)
        outputs = model_small(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
    
    

In [ ]:
# test the model after training on images test data
# print predictions and images


for i in range(len(test_data_small)):
    img = test_data_small.data[i]
    plt.imshow(img, cmap='gray')
    plt.show()
    img = img.reshape(-1, 28*28)
    img = img.type(torch.FloatTensor)
    output = model_small(img)
    _, predicted = torch.max(output.data, 1)
    print('Predicted: ', predicted.item())

In [ ]:
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
